# 智能重新排序 -- 使用大模型对每个检索到的块的相关性进行评分

智能重新排序的目的是帮助我们从海量文档中快速找到最相关的信息。它就像是给文档们排个队，让最有用的信息排在最前面。

### 工作原理
在RAG系统中，我们通常会先找到一些可能相关的文档，但这些文档的相关性可能不够精确。智能重新排序就是对这些文档进行再次评估，找出真正和我们的需求紧密相关的文档。

### 重排系统通常包括以下步骤：
- 初始检索器：使用向量搜索技术，找到一些可能相关的文档。
- 评分机制：给这些文档打分，看看它们和我们的需求有多匹配。
- 排序和选择逻辑：根据分数重新排序这些文档，挑选出最相关的几个。

首先，我们需要导入一些Python库，并从环境变量文件中加载必要的配置。

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["MINIMAX_GROUP_ID"] = os.getenv("MINIMAX_GROUP_ID")
os.environ["MINIMAX_API_KEY"] = os.getenv("MINIMAX_API_KEY")

from typing import List, Dict, Any, Tuple
from pydantic.v1 import BaseModel, Field

from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_qdrant import QdrantVectorStore
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import MiniMaxChat
from langchain_community.embeddings import MiniMaxEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

: 

#### 第一步：加载文档并提取文档内容
使用Docx2txtLoader类加载文档，这个类需要docx2txt库支持。

In [6]:
path = "../data/二十届三中全会.docx"
loader = Docx2txtLoader(path)
documents = loader.load()

#### 第二步：文本分块
将文本切割成小块

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
docs = text_splitter.split_documents(documents)
for doc in docs:
    doc.page_content = doc.page_content.replace('\t', ' ') 

#### 第三步：创建向量数据库
使用MiniMaxEmbeddings和QdrantVectorStore创建向量表示，并存储在本地的Qdrant数据库中。

In [8]:
embeddings = MiniMaxEmbeddings()
vectorstore = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url="http://localhost:6333/",
    prefer_grpc=True,
    collection_name="004"
)

#### 第四步：创建重排系统
创建一个重排系统，使用大模型来评估文档的相关性，并返回最相关的文档。

In [9]:
class RatingScore(BaseModel):
    relevance_score: float = Field(..., description="文档与提问的相关性得分")

In [10]:
def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    template = """
        在1-10的范围内，对以下文档与提问的相关性进行评分。考虑提问的特定上下文和目的，而不仅仅是关键字匹配。
        提问:{query}
        文档:{doc}
        相关性得分:
    """
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template=template
    )

    llm = ChatOpenAI(
        temperature=0,
        model="gpt-4o",
        openai_api_key=os.getenv("OPENAI_API_KEY"),
    )
    llm_chain = prompt_template | llm.with_structured_output(RatingScore)

    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        try:
            score = llm_chain.invoke(input_data)
            score = score.relevance_score
            score = float(score)
        except ValueError:
            score = 0  # 失败默认为0
        scored_docs.append((doc, score))
    
    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

#### 执行输出结果
最后，我们用一个实际的问题来测试我们的系统，看看它是否能够找到最相关的文档。

In [15]:
query = "什么是健全协商民主机制?"
initial_docs = vectorstore.similarity_search(query, k=15)
reranked_docs = rerank_documents(query, initial_docs)

# print first 3 initial documents
print("Top initial documents:")
for i, doc in enumerate(initial_docs[:3]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


# Print results
print(f"Query: {query}\n")
print("Top reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

得分: 9.0
得分: 2.0
得分: 2.0
得分: 3.0
得分: 3.0
得分: 2.0
得分: 3.0
得分: 3.0
得分: 2.0
得分: 2.0
得分: 2.0
得分: 3.0
得分: 2.0
得分: 2.0
得分: 2.0
Top initial documents:

Document 1:
（29）加强人民当家作主制度建设。坚持好、完善好、运行好人民代表大会制度。健全人大对行政机关、监察机关、审判机关、检察机关监督制度，完善监督法及其实施机制，强化人大预算决算审查监督和国有资产管理、政府债务管理监督。健全人大议事规则和论证、评估、评议、听证制度。丰富人大代表联系人民群众的内容和形式。健全吸纳民意、汇集民智工作机制。发挥工会、共青团、妇联等群团组织联系服务群众的桥梁纽带作用。

（30...

Document 2:
（33）深化立法领域改革。完善以宪法为核心的中国特色社会主义法律体系，健全保证宪法全面实施制度体系，建立宪法实施情况报告制度。完善党委领导、人大主导、政府依托、各方参与的立法工作格局。统筹立改废释纂，加强重点领域、新兴领域、涉外领域立法，完善合宪性审查、备案审查制度，提高立法质量。探索区域协同立法。健全党内法规同国家法律法规衔接协调机制。建设全国统一的法律法规和规范性文件信息平台。

（34）深入...

Document 3:
以习近平同志为核心的党中央团结带领全党全军全国各族人民，以伟大的历史主动、巨大的政治勇气、强烈的责任担当，冲破思想观念束缚，突破利益固化藩篱，敢于突进深水区，敢于啃硬骨头，敢于涉险滩，坚决破除各方面体制机制弊端，实现改革由局部探索、破冰突围到系统集成、全面深化的转变，各领域基础性制度框架基本建立，许多领域实现历史性变革、系统性重塑、整体性重构，总体完成党的十八届三中全会确定的改革任务，实现到党成立...
Query: 什么是健全协商民主机制?

Top reranked documents:

Document 1:
（29）加强人民当家作主制度建设。坚持好、完善好、运行好人民代表大会制度。健全人大对行政机关、监察机关、审判机关、检察机关监督制度，完善监督法及其实施机制，强化人大预算决算审查监督和国有资产管理、政府债务管理监督。健全人大议事规则和论证、评估、评议、听证制度。丰富人大代表联系人民群众的内容和形式